In [1]:
import numpy as np
import random
# import trimesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot
import copy
import os
import matplotlib.patches as mpatches
import sys
import time
import statistics



In [31]:

def cell(state,Size):
    if Size == 2:
        rand_cell = np.random.randint(0,3)#Random cell type
        if rand_cell == 0: #body centered
            state['CellType'].append('TR')
            state['Volume'].append(3.02)
        elif rand_cell == 1: #body centered
            state['CellType'].append('SC')
            state['Volume'].append(1.82)
        elif rand_cell == 2:  #simple cubic with rounded edges
            state['CellType'].append('FC')
            state['Volume'].append(3.69)
    elif Size == 1:
        rand_cell = np.random.randint(0,1)#Random cell type
        if rand_cell == 0:  #simple cubic with rounded edges
            state['CellType'].append('sol')
            state['Volume'].append(1)
        elif rand_cell == 1:  #simple cubic with rounded edges
            state['CellType'].append('sc')
            state['Volume'].append(0.7839)
        elif rand_cell == 2:  #simple cubic with rounded edges
            state['CellType'].append('bv')
            state['Volume'].append(0.7245)
    return state

In [3]:
def constraint(State,V_max):#Volume Constraint
    VOL = np.array(State['Volume']).reshape(-1,1)
    CELL =  np.array(State['CellType']).reshape(-1,1)
    POS = np.array(State['Position'])
    cell = CELL.flatten()
    V_sup = sum(VOL)
    if V_sup > V_max:
        ver = False
        print('Volume Violation:',V_sup,'>',V_max)
    else:
        ver = True
    return V_sup,ver

In [4]:
def initialize(Unit,Sup):
    T_state = {'CellType':[],'Volume':[],'Position':[],'Size':[]}
    for i in range(len(Unit)):
        new_cell = {'CellType':[],'Volume':[]}
        if i<len(Sup):
            size = 2
        else:
            size = 1
        new_cell = cell(new_cell,size)
        T_state['CellType'].append(new_cell['CellType'][0])
        T_state['Volume'].append(new_cell['Volume'][0])
        T_state['Position'].append(Unit[i].tolist())
        T_state['Size'].append(size)
    Vol,ver = constraint(T_state,0)#,0)
    return Vol,T_state

def evaluate(State,init,DisMax,StrMax):   #Call pyAnsys to evaluate objective and constraints
    num_cells=len(State['CellType'])
    CELL = np.array(State['CellType']).reshape(-1,1)
    POS = np.array(State['Position'])*1e-3
    Size = np.array(State['Size'])
    h =1e-3
    Dis, Stress, E = FEM_Thermal(CELL,POS,Size,h,init,DisMax,StrMax)   
    return E,init,Dis,Stress

In [5]:
def metopolis(E_prev,Vol,Vol2, new_state,prev_state,T,init,Dis,Str,DisMax,StrMax): #boolean of True/False
    E1 = E_prev
    Vol1=Vol
    Dis1 = Dis
    Str1 = Str
    E2,init,Dis2,Str2= evaluate(new_state,init,DisMax,StrMax)
    if E1 < E2: #E1 = Previous objective; E2 = Proposed objective
        P_acc = np.exp(-(E2-E1)/T) #minimize
        comp = np.random.uniform(0, 1)
        if P_acc > comp:
            accept = True
            E = E2
            Vol = Vol2
            Dis = Dis2
            Str = Str2
        else:
            accept = False
            P_acc = 0
            E = E1
            Vol = Vol1
            Dis = Dis1
            Str = Str1
    elif E2 == False:
        accept = False
        P_acc = 0
        E = E1
        Vol = Vol1
        Dis = Dis1
        Str = Str1
    else:
        P_acc = 1
        accept = True
        E = E2
        Vol = Vol2
        Dis = Dis2
        Str = Str2
    return accept,E, Vol, P_acc,Dis, Str

In [30]:
def ShapeAnneal(it,state,T,limit,n,alpha,Sup,Unit,E,Vol,init,V_max,Dis,Str,DisMax,StrMax):
    T_0 = T #Initial temperature
    P_acc = 1 #Probability of acceptance
    Res_loss = [] #initialize objective list for storage at each iteration
    V_iter = [] #initialize volume list for storage at each iteration
    Temp = [] #initialize temperature list for storage at each iteration
    P_acc_iter = [] #initialize probability of acceptance list for storage at each iteration
    Str_iter = [] #initialize p-Norm stress list for storage at each iteration
    Dis_iter = [] #initialize displacement list for storage at each iteration
    while it < 800:
        success = 0 
        for i in range(n):
            prev_state = copy.deepcopy(state)
            Total=len(Sup)
            Int = 150
            Tun = 400
            if i == 0:
                print('New Objective:',E)
            if T<=T_0 and T > Int:
                RANGE = int(Total)
            elif T>Tun and T<= Int:
                RANGE = int(Total/2)
            else:
                RANGE =int(Total/10)
            new_state = copy.deepcopy(state)
            for I in range (RANGE):
                Size = np.random.randint(1,3)
                if Size == 2:
                    rand_cell_select = np.random.randint(0,len(Sup)) #randomly select unit cell
                else:
                    rand_cell_select = np.random.randint(len(Sup),len(Unit)-len(Sup)) #randomly select unit cell
                new_cell = {'CellType':[],'Volume':[]} #initialize new cell
                new_cell = cell(new_cell,Size) #obtain new random cell
                new_state['CellType'][rand_cell_select]=new_cell['CellType'][0]
                new_state['Volume'][rand_cell_select]=new_cell['Volume'][0]
            Vol2,ver2 = constraint(new_state,V_max)
#                 print('constraint:',ver2)
            if ver2 == True:
                test,E_curr,Vol_curr, P_acc,Dis_curr, Str_curr = metopolis(E,Vol,Vol2,new_state,prev_state,T,init,Dis,Str,DisMax,StrMax)
                E = E_curr
                Vol = Vol_curr
                Dis = Dis_curr
                Str = Str_curr
#                     print('Objective',E)
            else:
                test = False

            it+=1 
            P_acc_iter.append(P_acc)
            Res_loss.append(E)
            V_iter.append(Vol)
            Str_iter.append(Str)
            Dis_iter.append(Dis)
#             print(P_acc)
            if test == True:
                state = new_state
                success += 1
            else:
                state = prev_state
#             print('Successes',success)

            if success > limit:
                print('~~~~~~~~New Temp~~~~~~~~~~')
                break
        if success == 0:
#             print('Success = 0')
            break
        T = T*alpha
    print('Final Objective',E,'Final Volume',np.round(Vol,3), 'Final Displacement', np.round(Dis,3), 'Final Stress', Str)
    print('Final State',state)
    return state,Res_loss,V_iter,P_acc_iter,Str_iter,Dis_iter


In [33]:
##### 
start_time = time.time()
T = 50 #initial temperature
n = 100 #number of mutations

limit = n/2 #maximum number of limits
alpha = 0.5 #cooling coefficient
np.set_printoptions(threshold=sys.maxsize)
CellCount = []
SOLU = []
start = 5
end = 15
for RUN in range(start,end):
    start_time = time.time()
    titleS = f"{os.getcwd()}\\AsymSupport.txt"
    Sup = np.loadtxt(titleS)

    Vol,state = initialize(Sup)

    %run ./AA_Adapter_MAPDL-Single.ipynb
    mapdl.exit()
    path = os.getcwd()
    mapdl = launch_mapdl(nproc = 8, run_location=path,override = True, license_type = "ansys")
    mapdl.clear()           
    
    print('state',state)
    DisMax = 66 #max displacement
    StrMax = 0.66 #max stress
    V_max = 4570 #max volume
    init = 0 #Indicator that this is the first time it is running
    E,init,Dis,Str= evaluate(state,init,DisMax,StrMax)
    init = 1 #Indicator that the iteration loop has begun
    print('Initial Objective',E,'Volume',np.round(Vol,3),'Max Displacement',np.round(Dis,3),'Max Stress',Str)
    CELL = np.array(state['CellType'])
    sc = np.count_nonzero(CELL == 'SC') #count number of SC
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bv')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    it = 1
    solutions,R_loss,V_iter,P_acc_iter,Str_iter,Dis_iter= ShapeAnneal(it,state,T,limit,n,alpha,Sup,Unit,E,Vol,init,V_max,Dis,Str,DisMax,StrMax)
    mapdl.exit()
    CELL = np.array(solutions['CellType'])
    SOLU.append(solutions)
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bv')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    print("--- %s seconds ---" % (time.time() - start_time))    
    Iter_data = np.concatenate((np.array(R_loss).reshape(-1,1),np.round(np.array(V_iter).reshape(-1,1),3),np.round(np.array(Str_iter).reshape(-1,1),3),np.round(np.array(Dis_iter).reshape(-1,1),3)),axis = 1)
    with open('ASymm3_%i' %RUN + '.txt', 'w') as f:
        content = str(Iter_data)
        f.write(content)
        f.close()
CC = np.array(CellCount)
with open('CellCount_ASymm3_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(CC)
    f.write(content)
    f.close()
with open('SOLU_ASymm_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(SOLU)
    f.write(content)
    f.close()

Volume Violation: [4360.] > 0
state {'CellType': ['TR', 'FC', 'SC', 'SC', 'SC', 'SC', 'FC', 'TR', 'FC', 'FC', 'TR', 'SC', 'TR', 'FC', 'TR', 'FC', 'TR', 'SC', 'FC', 'FC', 'FC', 'SC', 'TR', 'TR', 'FC', 'TR', 'TR', 'FC', 'TR', 'SC', 'FC', 'SC', 'TR', 'SC', 'SC', 'TR', 'SC', 'FC', 'TR', 'SC', 'SC', 'TR', 'TR', 'TR', 'FC', 'TR', 'SC', 'TR', 'TR', 'TR', 'SC', 'FC', 'SC', 'TR', 'SC', 'TR', 'TR', 'FC', 'FC', 'TR', 'TR', 'TR', 'TR', 'FC', 'TR', 'SC', 'FC', 'SC', 'TR', 'SC', 'SC', 'FC', 'TR', 'TR', 'TR', 'SC', 'SC', 'FC', 'FC', 'FC', 'TR', 'FC', 'SC', 'TR', 'FC', 'SC', 'TR', 'SC', 'SC', 'TR', 'SC', 'FC', 'FC', 'FC', 'FC', 'TR', 'SC', 'SC', 'FC', 'SC', 'FC', 'FC', 'SC', 'FC', 'FC', 'SC', 'FC', 'FC', 'FC', 'FC', 'TR', 'SC', 'SC', 'SC', 'TR', 'TR', 'SC', 'FC', 'SC', 'SC', 'SC', 'FC', 'SC', 'TR', 'FC', 'SC', 'SC', 'TR', 'FC', 'SC', 'FC', 'FC', 'TR', 'TR', 'FC', 'FC', 'FC', 'FC', 'TR', 'FC', 'TR', 'FC', 'FC', 'FC', 'FC', 'TR', 'FC', 'TR', 'TR', 'TR', 'TR', 'SC', 'FC', 'FC', 'TR', 'TR', 'FC', 'SC', 'F

Structural boundary conditions attached--- 1.0740010738372803 seconds ---
Initial Objective [-3906.52261594] Volume [4360.] Max Displacement 13.207 Max Stress 0.11045999882631317
Simple Cubic: 210 Transition Cubic: 207 Face-Centered Vertile Cubic: 214
SC1: 0 BV1 Cubic: 0 Solid Cubic: 2563
New Objective: [-3906.52261594]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3907.19260821]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3893.19666828]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-4027.97692965]
New Objective: [-4142.0069462]
New Objective: [-4219.37260793]
New Objective: [-4281.29722477]
New Objective: [-4353.65442578]
New Objective: [-4368.58321703]
Final Objective [-4379.39368846] Final Volume [4508.18] Final Displacement 12.538 Final Stress 0.10683163848229726
Final State {'CellType': ['FC', 'FC', 'FC', 'FC', 'FC', 'FC', 'SC', 'SC', 'FC', 'FC', 'TR', 'FC', 'SC', 'TR', 'FC', 'FC', 'TR', 'FC', 'SC', 'SC', 'TR', 'FC', 'FC', 'SC', 'FC', 'SC', 'SC', 'FC', 'TR', 'FC', 'FC', 'FC', 'FC', 'FC

Simple Cubic: 139 Transition Cubic: 184 Face-Centered Vertile Cubic: 308
SC1: 0 BV1 Cubic: 0 Solid Cubic: 2563
--- 12957.841128110886 seconds ---
Volume Violation: [4344.2] > 0
state {'CellType': ['FC', 'SC', 'FC', 'FC', 'TR', 'TR', 'TR', 'TR', 'TR', 'FC', 'TR', 'SC', 'FC', 'SC', 'SC', 'TR', 'FC', 'TR', 'TR', 'SC', 'SC', 'TR', 'TR', 'SC', 'TR', 'FC', 'FC', 'FC', 'TR', 'TR', 'FC', 'TR', 'SC', 'FC', 'TR', 'SC', 'TR', 'FC', 'FC', 'TR', 'TR', 'SC', 'TR', 'SC', 'SC', 'TR', 'FC', 'FC', 'SC', 'FC', 'TR', 'FC', 'SC', 'TR', 'SC', 'SC', 'FC', 'SC', 'TR', 'SC', 'TR', 'TR', 'TR', 'SC', 'SC', 'SC', 'TR', 'FC', 'TR', 'SC', 'FC', 'TR', 'FC', 'TR', 'SC', 'FC', 'FC', 'FC', 'FC', 'FC', 'TR', 'SC', 'TR', 'FC', 'FC', 'FC', 'SC', 'FC', 'FC', 'TR', 'FC', 'TR', 'SC', 'TR', 'TR', 'TR', 'FC', 'TR', 'SC', 'SC', 'SC', 'SC', 'SC', 'TR', 'FC', 'SC', 'TR', 'FC', 'TR', 'FC', 'TR', 'FC', 'FC', 'SC', 'TR', 'FC', 'FC', 'TR', 'TR', 'FC', 'TR', 'SC', 'SC', 'TR', 'TR', 'TR', 'SC', 'FC', 'SC', 'TR', 'TR', 'FC', 'TR', 'FC',

Structural boundary conditions attached--- 1.1490013599395752 seconds ---
Initial Objective [-3889.25985094] Volume [4344.2] Max Displacement 13.309 Max Stress 0.10777805908752391
Simple Cubic: 212 Transition Cubic: 225 Face-Centered Vertile Cubic: 194
SC1: 0 BV1 Cubic: 0 Solid Cubic: 2563
New Objective: [-3889.25985094]


KeyboardInterrupt: Interrupted during MAPDL execution